In [132]:
%%writefile deployment/deploy.py

from azureml.core import Image, Workspace, Webservice, Model
from azureml.core.webservice import AciWebservice
from azureml.core.authentication import ServicePrincipalAuthentication
import time

def service_principal_auth(tenant_id, sp_id, sp_secret):
  return ServicePrincipalAuthentication(
      tenant_id=tenant_id,
      service_principal_id=sp_id,
      service_principal_password=sp_secret)

time.sleep(120)

service_principal_clientid = '74809eff-c1cd-4b4d-be93-1485502de2cf'
service_principal_secret ='-XPEtO1~nKVM92F_1jYEVH_8slf3Mmh16Y'
      
ws = Workspace.get(name="sparkml",
               subscription_id='f3f672c1-6cfc-4f72-92ae-2b1ab1c0cf69',
               resource_group='sparkdemorg',
               auth=service_principal_auth('72f988bf-86f1-41af-91ab-2d7cd011db47', service_principal_clientid , service_principal_secret))

# Remove existing services
for svc in Webservice.list(ws,image_name="km-predictor-image"):
    svc.delete()

# Get most recent model
recent_model_version = 0
recent_model = None
for m in Model.list(ws, name='km-predictor-model'):
    if recent_model_version < m.serialize()['version']:
        recent_model_version = m.serialize()['version']
        recent_model = m
        print(m)

recent_image = None
while recent_image == None:
    images = Image.list(ws, image_name='km-predictor-image')

    # Get most recent version of image
    recent_version = 0
    recent_image = None
    for i in images:
        if recent_model_version == i.serialize()['version'] and i.serialize()['creationState'] == 'Succeeded':
            recent_image = i
            print(i)
    time.sleep(60)


# Deploy webservice
print("Deploying web service")
webservice_deployment_config = AciWebservice.deploy_configuration()
webservice = Webservice.deploy_from_image(
                            workspace=ws, 
                            name="km-predictor-service", 
                            image=recent_image, 
                            deployment_config=webservice_deployment_config)

webservice.wait_for_deployment()

Overwriting deployment/deploy.py


In [133]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.widgets import RunDetails
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)



aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")


SDK version: 1.31.0
found existing compute target.
Azure Machine Learning Compute attached


In [134]:
ws = Workspace.from_config()
source_directory = './deployment'
step1 = PythonScriptStep(name="deploy_step",
                         script_name="deploy.py", 
                         compute_target=aml_compute, 
                         source_directory=source_directory,
                         allow_reuse=False)
pipeline1 = Pipeline(workspace=ws, steps=[step1])

pipeline_run = Experiment(ws, 'deployment-exp').submit(pipeline1, regenerate_outputs=False)

published_pipeline1 = pipeline_run.publish_pipeline(
     name="My_Published_Pipeline",
     description="My Published Pipeline Description",
     version="1.0")

published_pipeline1

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


Created step deploy_step [7c5c104f][2edf8af9-25bc-404b-a18e-c15becf0c9e5], (This step will run and generate new outputs)
Submitted PipelineRun 2f1c966a-a4af-4f5b-9574-ac5506882f45
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2f1c966a-a4af-4f5b-9574-ac5506882f45?wsid=/subscriptions/f3f672c1-6cfc-4f72-92ae-2b1ab1c0cf69/resourcegroups/sparkdemorg/workspaces/sparkml&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


Name,Id,Status,Endpoint
My_Published_Pipeline,bb6d8fd2-f82e-4d4f-815c-ac0c85122796,Active,REST Endpoint
